In [30]:
# "Dataset_03.csv"

<img src="Dataset_03.png" height="350" width="500">

In [31]:
import pandas as pd
df = pd.read_csv("Dataset_03.csv")
df.head(2)

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female


### 1. 이마의폭(forehead_width_cm) 과 높이(forehead_height_cm) 사이의 비율(forehead_ratio) 에 대해서 평균으로부터 3 표준편차 밖의 경우를 이상치로 정의할 때 , 이상치에 해당하는 데이터는 몇 개인가 ?
(답안 예시) 10

In [32]:
df["forehead_ratio"] = df["forehead_width_cm"] / df["forehead_height_cm"]
df.head(2)

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender,forehead_ratio
0,1,11.8,6.1,1,0,1,1,Male,1.934426
1,0,14.0,5.4,0,0,1,0,Female,2.592593


In [33]:
stat_mean = df["forehead_ratio"].mean()
stat_std  = df["forehead_ratio"].std()
limit_lower = stat_mean - (3 * stat_std)
limit_upper = stat_mean + (3 * stat_std)

In [34]:
# df_out = df.loc[() | (), ]
df_out = df.loc[(df["forehead_ratio"] < limit_lower) | (df["forehead_ratio"] > limit_upper), ]
len(df_out)

3

In [35]:
sum((df["forehead_ratio"] < limit_lower) | (df["forehead_ratio"] > limit_upper))

3

### 2. 성별에 따라 forehead_ratio 평균에 차이가 있는지 적절한 통계 검정을 수행하시오
- 검정은 이분산을 가정하고 수행한다
- 검정통계량의 추정치를 소수점 셋째 자리까지 반올림하여 기술하시오
- 신뢰수준 99% 에서 양측 검정을 수행하고 결과는 귀무가설 기각의 경우 Y 로 , 그렇지 않을 경우 N 으로 답하시오. 
(답안 예시) 1.234, Y

In [36]:
from scipy.stats import ttest_ind

In [37]:
df["gender"].unique()

array(['Male', 'Female'], dtype=object)

In [38]:
stat, p = ttest_ind(df.loc[df["gender"] == "Male",   "forehead_ratio"],
                    df.loc[df["gender"] == "Female", "forehead_ratio"], 
                    equal_var = False)
print(round(stat, 3)) # <-- 2.999
print(round(p, 3)) # <-- Y

2.999
0.003


### 3. 주어진 데이터를 사용하여 성별을 구분할 수 있는지 로지스틱 회귀분석을 적용하여 알아 보고자 한다 .
- 데이터를 7 대 3 으로 나누어 각각 Train 과 Test set 로 사용한다 . 이 때 seed 는 123 으로 한다.
- 원 데이터에 있는 7 개의 변수만 Feature 로 사용하고 gender 를 label 로 사용한다.

(forehead_ratio 는 사용하지 않음)

- sklearn 라이브러리의 LogisticRegression() 함수를 사용하시오.
- 로지스틱 회귀분석 예측 함수와 Test dataset 를 사용하여 예측을 수행하고 정확도를 평가한다. 이 때 임계값은 0.5 를 사용한다 .
- Male 의 Precision 값을 소수점 둘째 자리까지 반올림하여 기술하시오.

(답안 예시) 0.12

In [39]:
from sklearn.model_selection import train_test_split
df["gender"] = (df["gender"] == "Male") + 0
df_train, df_test = train_test_split(df.iloc[:, :-1], # 마지막 변수 제외
                                     train_size = 0.7, 
                                     random_state = 123)
print(len(df_train))
print(len(df_test))

3500
1501


In [40]:
df.head(2)

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender,forehead_ratio
0,1,11.8,6.1,1,0,1,1,1,1.934426
1,0,14.0,5.4,0,0,1,0,0,2.592593


In [41]:
df_train.head(2)

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
3991,1,12.5,6.9,1,1,1,0,1
4163,1,12.4,5.3,1,1,1,1,1


In [42]:
from sklearn.linear_model import LogisticRegression

In [46]:
model = LogisticRegression().fit(X = df_train.drop("gender", axis = 1),
                                 y = df_train["gender"])

In [44]:
pred = model.predict_proba(df_test.drop("gender", axis = 1))
pred[:4, ] # 첫 번째 column은 0이될 확률, 두 번째 column은 1이될 확률

array([[2.13366044e-02, 9.78663396e-01],
       [8.97496054e-05, 9.99910250e-01],
       [2.54440411e-01, 7.45559589e-01],
       [2.15025315e-03, 9.97849747e-01]])

In [45]:
pred[:4, 1]

array([0.9786634 , 0.99991025, 0.74555959, 0.99784975])

In [30]:
pred_class = (pred[:, 1] > 0.5) + 0
pred_class[:4]

array([1, 1, 1, 1])

In [31]:
from sklearn.metrics import precision_score

In [33]:
round(precision_score(y_true = df_test["gender"],
                      y_pred = pred_class), 2)

0.96